# Adding connectivity motifs

This is a simple way to increase the number of connectivity motifs in a randomly connected matrix. It does not give you close control of the amount of increase -- for example, reciprical motifs are increased more than convergent/divergent motifs. However, it is simple and fast! Note that I only track reciprical, convergent, and divergent motifs (not chain motifs).

The basic idea is to reserve a set of core neurons that will have more connections than the remaining neurons.

<img src="img/motifs.png" width=500>

Image from [Synchronization from second order network connectivity statistics](https://www.frontiersin.org/articles/10.3389/fncom.2011.00028/full) by Zhao, Beverlin, Netoff, and Nykamp (2011), which gives an explanation how to create these motifs with more control.

In [2]:
import numpy as np
import numpy.random as npr
from scipy import sparse
from scipy.spatial.distance import cosine
from time import time

from matplotlib import pyplot as plt
from matplotlib import animation, rc, collections
from IPython.display import HTML

np.set_printoptions(precision=3)

In [3]:
def get_weights_rand(shape, prob):
    """Return a weights matrix with prob amount of random connections activated.
    
    Args:
        shape: (int) number of neurons in network
        prob: (float) fraction of connections to be activated
    
    Returns:
        weights: (numpy matrix) connectivity weight matrix
    """
    # THIS IS NOT QUITE RANDOM...
    l = int(shape*shape*prob)
    rows = npr.choice(range(shape), l)
    cols = npr.choice(range(shape), l)
    weights = sparse.csr_matrix((np.ones(l), (rows, cols)), shape=(shape,shape))
    weights.setdiag(0)
    return weights

def get_weights_motifs(shape, prob, split=.5, core=.5):
    """Return a weights matrix with prob amount of random connections activated.
    
    Args:
        shape: (int) number of neurons in network
        prob: (float) fraction of connections to be activated
    
    Returns:
        weights: (numpy matrix) connectivity weight matrix
    """
    # THIS IS NOT QUITE RANDOM...
    l = int(shape*shape*prob)
    l_split = int(l * split)
    shape_split = int(shape * core)
    rows_core = npr.choice(range(shape_split), l_split)
    cols_core = npr.choice(range(shape_split), l_split)
    rows_all = npr.choice(range(shape), l - l_split)
    cols_all = npr.choice(range(shape), l - l_split)
    rows = np.concatenate((rows_core, rows_all))
    cols = np.concatenate((cols_core, cols_all))
    weights = sparse.csr_matrix((np.ones(len(rows)), (rows, cols)), shape=(shape,shape))
    weights.setdiag(0)
    return weights

def count_recip(mat):
    """Return number of edges in a recip motif."""
    t = mat.transpose()
    recips = mat.multiply(t)
    return recips.count_nonzero() / 2

def count_conv(mat):
    """Return number of edges in a conv motif."""
    row = np.array(mat.sum(axis=0))[0]
    motifs = row[np.where(row > 1)]
    count = [int(n * (n + 1) / 2) for n in motifs]
    return sum(count)

def count_div(mat):
    """Return number of edges in a div motif."""
    col = np.array(mat.sum(axis=1)).transpose()[0]
    motifs = col[np.where(col > 1)]
    count = [int(n * (n + 1) / 2) for n in motifs]
    return sum(count)

def sizeof_fmt(num, suffix=''):
    for unit in ['','thousand','million']:
        if abs(num) < 1000.0:
            return "%3.1f %s %s" % (num, unit, suffix)
        num /= 1000.0
    return "%.1f %s %s" % (num, 'trillion', suffix)

def check_motifs(n, p, sp, cr):
    print('n:', n, 'p:', p)
    print('split:', sp, 'core:', cr)

    start = time()
    W = get_weights_motifs(n, p, split=sp, core=cr) # connectivity weight matrix; are neurons connected?
    print('made W in {:.2f} seconds'.format(time() - start))
    print('numb edges:', sizeof_fmt(W.count_nonzero()))

    start = time()
    num_recips = count_recip(W)
    print('\ncounted recips in {:.2f} seconds'.format(time() - start))
    print('numb recip:', sizeof_fmt(num_recips))
    expd = p*p*n*(n-1) / 2
    print('expd recip:', sizeof_fmt(expd))
    print('increase: {:.2f}x'.format(num_recips / expd))

    start = time()
    num_conv = count_conv(W)
    print('\ncounted conv in {:.2f} seconds'.format(time() - start))
    print('numb conv:', sizeof_fmt(num_conv))
    expd = p*p*n*(n-1)*(n-2) / 2
    print('expd conv:', sizeof_fmt(expd))
    print('increase: {:.2f}x'.format(num_conv / expd))

    start = time()
    num_div = count_div(W)
    print('\ncounted div in {:.2f} seconds'.format(time() - start))
    print('numb div:', sizeof_fmt(num_div))
    print('expd div:', sizeof_fmt(expd))
    print('increase: {:.2f}x'.format(num_div / expd))

n = 100000 # number of neurons in network; CA3 is 330,000
p = 0.01 # connection probability for weight matrix

sp = .9 # fraction of connections to reserve for core group
cr = .4 # fraction of neurons in core group

check_motifs(n, p, sp, cr)

n: 100000 p: 0.01
split: 0.9 core: 0.4


/anaconda3/lib/python3.6/site-packages/scipy/sparse/compressed.py:746: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


made W in 41.10 seconds
numb edges: 97.4 million 

counted recips in 6.56 seconds
numb recip: 2.5 million 
expd recip: 500.0 thousand 
increase: 4.96x

counted conv in 0.25 seconds
numb conv: 110.8 trillion 
expd conv: 50.0 trillion 
increase: 2.22x

counted div in 0.11 seconds
numb div: 110.8 trillion 
expd div: 50.0 trillion 
increase: 2.22x
